<a href="https://colab.research.google.com/github/AlexeyTri/MLSeminars/blob/main/MLHSE/Seminar5_MLDS_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Отбор признаков

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import sklearn

In [ ]:
!pip install category_encoders

In [ ]:
import category_encoders

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
np.random.seed(42)

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5%205/Pokemon.csv')

In [ ]:
data

In [ ]:
data.isna().sum()
# data['Type 2'] = data['Type 2'].fillna('No second type')

data.drop(columns=['#', 'Name'], inplace=True)

In [ ]:
X = data.drop('Legendary', axis=1)
y = data['Legendary'].astype('int')

In [ ]:
y.value_counts(normalize=True)

0    0.91875
1    0.08125
Name: Legendary, dtype: float64

Make some default pipline

In [ ]:
X

,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,Grass,Poison,318,45,49,49,65,65,45,1
1,Grass,Poison,405,60,62,63,80,80,60,1
2,Grass,Poison,525,80,82,83,100,100,80,1
3,Grass,Poison,625,80,100,123,122,120,80,1
4,Fire,No second type,309,39,52,43,60,50,65,1
...,...,...,...,...,...,...,...,...,...,...
795,Rock,Fairy,600,50,100,150,100,150,50,6
796,Rock,Fairy,700,50,160,110,160,110,110,6
797,Psychic,Ghost,600,80,110,60,150,130,70,6
798,Psychic,Dark,680,80,160,60,170,130,80,6


In [ ]:
cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)), 
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))
])

In [ ]:
cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [ ]:
cv_res1 = cross_validate(default_pipeline,
                         X,
                         y,
                         cv=5,
                         scoring='f1',                         
                         n_jobs=-1,
                         return_train_score=True
                         )

In [ ]:
cv_res1['test_score'].mean()

0.5466128466128467

Make pipeline more complicated

In [ ]:
pip_diff = Pipeline([
    ('cat_encder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('polyfeatures_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))
    ])

In [ ]:
cv_res2 = cross_validate(pip_diff,
                          X,
                          y,
                          cv=5,
                          scoring='f1',
                          n_jobs=-1,
                          return_train_score=True)

In [ ]:
cv_res2

{'fit_time': array([0.06695437, 0.07403207, 0.07642555, 0.09207535, 0.08109736]),
 'score_time': array([0.01578903, 0.01665235, 0.01390171, 0.03237462, 0.01130939]),
 'test_score': array([0.375     , 0.88888889, 0.5       , 0.66666667, 0.53658537]),
 'train_score': array([0.95145631, 0.89583333, 0.97142857, 0.96153846, 0.98076923])}

In [ ]:
cv_res2['train_score'].mean()

0.9522051815498418

In [ ]:
cv_res2['test_score'].mean()

0.5934281842818427

Introduce feature selectors

In [ ]:
data_tr = pip_diff[:-1]

In [ ]:
X_tr = data_tr.fit_transform(X, y)
X_tr.shape, y.shape

((800, 1001), (800,))

Фильтрационные методы

In [ ]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [ ]:
pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selecetor_', SelectKBest(score_func=mutual_info_classif, k=100)),
    ('model_', SVC(kernel='linear'))

])

In [ ]:
cv_res = cross_validate(pipe,X,y,cv=5, scoring='f1', return_train_score=True)

In [ ]:
cv_res['test_score'].mean()

0.4433766233766233

In [ ]:
cv_res['test_score'].mean()

In [ ]:
cv_res['train_score'].mean()

0.7823530494795374

Жадный метод отбора

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
k_best = 50

rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)

In [ ]:
X_tr.shape

(800, 1001)

In [ ]:
res = rfe.fit_transform(X_tr, y)
res.shape

In [ ]:
pipe_rfe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selecetor_', RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)),
    ('model_', SVC(kernel='linear'))

])

In [ ]:
cv_res = cross_validate(pipe_rfe,X,y,cv=5, scoring='f1', return_train_score=True)

In [ ]:
cv_res['test_score'].mean(), cv_res['train_score'].mean()

(0.6551901108269395, 0.91305189625414)

С помощью модели

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
sel = SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), threshold=0.1)

In [ ]:
sel = sel.fit_transform(X_tr, y)
sel.shape

(800, 37)

In [ ]:
pipe_sel = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selecetor_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), threshold=0.1)),
    ('model_', SVC(kernel='linear'))

])

In [ ]:
cv_sel = cross_validate(pipe_sel,X,y,cv=5, scoring='f1', return_train_score=True)

In [ ]:
cv_sel['test_score'].mean(), cv_sel['train_score'].mean()

(0.6171946003742315, 0.9186863636419137)

PCA или метод главных компонент

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

#X = [[1,3],[0,2],[0,0],[3,3]]
X = [[0,1],[-1,0],[-1,-2],[2,1]]
pca.fit_transform(X)

array([[-0.70710678,  0.70710678],
       [ 0.70710678,  0.70710678],
       [ 2.12132034, -0.70710678],
       [-2.12132034, -0.70710678]])

In [ ]:
pca.explained_variance_ratio_

array([0.83333333, 0.16666667])

In [ ]:
pca = PCA(n_components = 15)

In [ ]:
res = pca.fit_transform(X_tr)
display(res.shape)
res

(800, 2)

array([[-19.80072907,   4.46072632],
       [-15.87855227,   3.10134435],
       [ -6.24072086,  -0.46328011],
       ...,
       [ 32.67677451,  15.03404521],
       [ 55.33222614,  14.85637261],
       [ 19.98280532,  -4.60646431]])

In [ ]:
# суммарная доля объясненной дисперсии исходных признаков

pca.explained_variance_ratio_.sum()

0.5743580120889531

In [ ]:
# каждая следующая компонента менее информативна чем предыдущая
pca.explained_variance_ratio_

array([0.45670112, 0.1176569 ])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_components = 15

pipe_pca = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', PCA(n_components=n_components)), 
    ('model_', RandomForestClassifier())])

cv_res5 = cross_validate(pipe_pca, X, y, cv=5, scoring='roc_auc', return_train_score=True)
cv_res5

{'fit_time': array([0.31144643, 0.2890532 , 0.30890322, 0.30893946, 0.32903051]),
 'score_time': array([0.02441812, 0.02052093, 0.02226686, 0.02073693, 0.02190876]),
 'test_score': array([0.95421245, 0.9714809 , 0.96153846, 0.93799058, 0.93851387]),
 'train_score': array([0.99926413, 0.99893707, 0.99885531, 0.99895343, 0.99914966])}

In [ ]:
cv_res5['test_score'].mean(), cv_res5['train_score'].mean()

(0.9527472527472527, 0.9990319204604919)